In [ ]:
import numpy as np
import os
import torch
import importlib
import data_set

importlib.reload(data_set)
#加载图片相关数据
data = data_set.ImageSet()
cuda_ok = torch.cuda.is_available()
print("Cuda is available" if cuda_ok else "There is no gpu available.")
use_cuda = False & cuda_ok

In [ ]:
#加载一些预先训练好的model
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
squeezenet = models.squeezenet1_0(pretrained=True)
vgg16 = models.vgg16(pretrained=True)
densenet = models.densenet161(pretrained=True)
inception = models.inception_v3(pretrained=True)
googlenet = models.googlenet(pretrained=True)
shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
mobilenet = models.mobilenet_v2(pretrained=True)
resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
wide_resnet50_2 = models.wide_resnet50_2(pretrained=True)
mnasnet = models.mnasnet1_0(pretrained=True)


In [ ]:
images,lables = data.GetBatch([0,1,2,6])
#直接使用CPU来进行预测运算
with torch.no_grad():
    ret = googlenet(images)
print("lables:")
print(data.category_names[lables])
print("predictions:")
print(data.category_names[ret.topk(1)[1]])

    

In [ ]:
#使用fgsm来生成所有的攻击图片
import fgsm
importlib.reload(fgsm)
attack_ret = []
model = squeezenet
model = model.cuda() if use_cuda else model.cpu()
for i in range(data.__len__()):
    image,label = data.__getitem__(i)
    r = fgsm.attack(model, image, label, 0.01 )
    attack_ret.append(r)


In [ ]:
#show result
id = 5
image, label = data.__getitem__(id)
print('label is %d "%s"' % (label, data.category_names[label]))
print(
    'origin:\nprediction is %d(%f) "%s", \nlabel probability: %f'
    % (attack_ret[1][0], attack_ret[1][1], data.category_names[attack_ret[1][0]], attack_ret[1][2])
)
plt.figure()
plt.imshow(tensor2numpy(image))
plt.show()
print(
    'attack result:\nprediction is %d(%f) "%s", \nlabel probability: %f'
    % (attack_ret[2][0], attack_ret[2][1], data.category_names[attack_ret[2][0]], attack_ret[2][2])
)